#### Ejercicio 6.1 imagen Saigon 

In [17]:
import numpy as np
from PIL import Image

def histogram_equalization(image_array):
    # Flatten the image array and calculate the histogram
    histogram, _ = np.histogram(image_array.flatten(), bins=256, range=[0,256])

    # Calculate the cumulative distribution function
    cdf = histogram.cumsum()
    cdf_normalized = cdf * histogram.max() / cdf.max()

    # Use the cumulative distribution function to map the old pixel values to new ones
    cdf_m = np.ma.masked_equal(cdf, 0)
    cdf_m = (cdf_m - cdf_m.min()) * 255 / (cdf_m.max() - cdf_m.min())
    cdf = np.ma.filled(cdf_m, 0).astype('uint8')

    # Map the old values to the new values
    image_equalized = cdf[image_array]
    
    return image_equalized

# Load the image and convert to grayscale
saigon_image_path = './Imgs//saigon.jpeg'  # Update with the correct path
saigon_image = Image.open(saigon_image_path).convert('L')
saigon_array = np.array(saigon_image)

# Apply histogram equalization
saigon_equalized_array = histogram_equalization(saigon_array)

# Convert back to PIL image and save
saigon_equalized_image = Image.fromarray(saigon_equalized_array)
saigon_equalized_image.save('saigon_equalized.jpeg')


#### Ejercicio 6.2 imagen Monkey

In [15]:
import numpy as np
from PIL import Image

def histogram_equalization(image_array):
    # Flatten the image array and calculate the histogram
    histogram, _ = np.histogram(image_array.flatten(), bins=256, range=[0,256])

    # Calculate the cumulative distribution function
    cdf = histogram.cumsum()
    cdf_normalized = cdf * histogram.max() / cdf.max()

    # Use the cumulative distribution function to map the old pixel values to new ones
    cdf_m = np.ma.masked_equal(cdf, 0)
    cdf_m = (cdf_m - cdf_m.min()) * 255 / (cdf_m.max() - cdf_m.min())
    cdf = np.ma.filled(cdf_m, 0).astype('uint8')

    # Map the old values to the new values
    image_equalized = cdf[image_array]
    
    return image_equalized

# Load the image and convert to grayscale
monkey_image_path = './imgs/monkey.jpeg'  # Update with the correct path
monkey_image = Image.open(monkey_image_path).convert('L')
monkey_array = np.array(monkey_image)

# Apply histogram equalization
monkey_equalized_array = histogram_equalization(monkey_array)

# Convert back to PIL image and save
monkey_equalized_image = Image.fromarray(monkey_equalized_array)
monkey_equalized_image.save('monkey_equalized.jpeg')


#### Ejercicio 6.3 imagen Rice

In [11]:
from PIL import Image
import numpy as np

# Load the image
rice_image_path = './Imgs/rice.jpg'
rice_image = Image.open(rice_image_path).convert('L')  # Convert to grayscale

# Convert to NumPy array
rice_array = np.array(rice_image)
print(rice_array)

# Apply manual thresholding
threshold = 130  # You may need to adjust this threshold
rice_segmented = rice_array > threshold

# Convert back to PIL image and save
segmented_image = Image.fromarray(np.uint8(rice_segmented) * 255, 'L')
segmented_image.save('rice_segmented.jpg')

# Display the segmented image
segmented_image.show()


[[127 126 121 ...  95  93  91]
 [125 124 120 ...  95  93  91]
 [122 121 118 ...  93  93  91]
 ...
 [ 57  57  56 ...  46  46  46]
 [ 59  58  57 ...  43  42  42]
 [ 61  60  59 ...  42  41  41]]
